# Transfer Learning with CIFAR-10 and ResNet50

This Jupyter notebook demonstrates the use of transfer learning to classify images from the CIFAR-10 dataset using the ResNet50 model. Transfer learning is a technique where a pre-trained model is used as a feature extractor for a new task. In this case, we are using the ResNet50 model, which has been pre-trained on the ImageNet dataset, to extract features from the CIFAR-10 images.

## Import Libraries

We start by importing the necessary libraries.

```python
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import ResNet50
```

## Load and Preprocess Data

We load the CIFAR-10 dataset and normalize the pixel values to be between 0 and 1.

```python
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
```

We then create `tf.data.Dataset` objects for the training and test data, shuffle and batch the training data, and batch the test data.

```python
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(32)
```

## Define the Model

We define the model architecture using the ResNet50 model as a feature extractor and a custom classifier.

### Feature Extractor

The feature extractor is defined as a function that takes the input images and returns the output of the ResNet50 model with the top layers removed.

```python
def feature_extractor(inputs):
    feature_extractor = ResNet50(input_shape=(224, 224, 3),
                                 include_top=False,
                                 weights='imagenet')(inputs)
    return feature_extractor
```

### Classifier

The classifier is defined as a function that takes the output of the feature extractor and returns the final classification output. It consists of a global average pooling layer, two dense layers with ReLU activation, and a final dense layer with softmax activation.

```python
def classifier(inputs):
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(10, activation="softmax")(x)
    return x
```

### Final Model

The final model is defined as a function that takes the input images, resizes them to 224x224, passes them through the feature extractor, and then through the classifier.

```python
def final_model(inputs):
    resize = layers.UpSampling2D(size=(7,7))(inputs)
    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)
    return classification_output
```

We then create the model using the `tf.keras.models.Model` class and compile it using the Adam optimizer and sparse categorical cross-entropy loss.

```python
inputs = layers.Input(shape=(32, 32, 3))
outputs = final_model(inputs)
model = models.Model(inputs, outputs)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
```

## Train the Model

Finally, we train the model using the training data and validate it using the test data.

```python
model.fit(train_dataset, epochs=1, validation_data=test_dataset)
```

Transfer Learning with CIFAR-10 and ResNet50

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import ResNet50

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Create a tf.data.Dataset object for the training data
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

# Shuffle and batch the training data
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

# Create a tf.data.Dataset object for the test data
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# Batch the test data
test_dataset = test_dataset.batch(32)

# Define the feature extractor
def feature_extractor(inputs):
    feature_extractor = ResNet50(input_shape=(224, 224, 3),
                                 include_top=False,
                                 weights='imagenet')(inputs)
    return feature_extractor

# Define the classifier
def classifier(inputs):
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(10, activation="softmax")(x)
    return x

# Define the model
def final_model(inputs):
    resize = layers.UpSampling2D(size=(7,7))(inputs)
    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)
    return classification_output

# Create the model
inputs = layers.Input(shape=(32, 32, 3))
outputs = final_model(inputs)
model = models.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=1, validation_data=test_dataset)


94765736/94765736 [==============================] - 5s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1563/1563 [==============================] - 584s 348ms/step - loss: 0.9034 - accuracy: 0.6882 - val_loss: 0.9700 - val_accuracy: 0.6730
